In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

Info    : Increasing process stack size (8192 kB < 16 MB)


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000524251s, CPU 0.000526s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00875961s, CPU 0.007683s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing curve 3 order 2
Info    : [ 70%] Meshing curve 4 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.781163
Info    : Done meshing order 2 (Wall 0.000885342s, CPU 0.000854s)
Info    : 357 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357)

In [4]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [5]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [6]:
FEM.initialTemperature!("left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [7]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.020833333333333297; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [8]:
FEM.applyBoundaryConditions!(K, C, q, [supp])

In [9]:
λmax = FEM.largestEigenValue(K, C)

161355.0686209338

In [10]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.002581667626896283 … -0.8020336718558312 1.3395259667126231; … ; 0.0 1.7230050428804905e-6 … -0.049873545587967305 0.10904683979677808; 0.0 3.620862210175205e-6 … 0.058314726384816225 -0.07355993797795542], [0.0, 0.0003645595388760728, 0.0007291190777521456, 0.0010936786166282184, 0.0014582381555042912, 0.001822797694380364, 0.0021873572332564368, 0.0025519167721325094, 0.002916476311008582, 0.0032810358498846547, 0.0036455953887607273, 0.0040101549276368, 0.004374714466512873, 0.004739274005388945, 0.005103833544265018, 0.005468393083141091, 0.005832952622017163, 0.006197512160893236, 0.0065620716997693085, 0.006926631238645381], Int64[], 20, :T, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [11]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.010077163895915514 … 0.0219495172755809 0.02202453310540394; … ; 0.0 0.0035203738398650953 … 0.01502578822207357 0.01509204317681318; 0.0 0.003912263265401068 … 0.01571077665622408 0.01577868572289641], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [12]:
FEM.showDoFResults(T, :T, visible=true)

1

In [13]:
q1 = FEM.solveHeatFlux(T)

LowLevelFEM.VectorField([[0.0 -0.6806420869613321 … -0.9930046686000746 -0.9949419666513158; 0.0 -0.001327830952720932 … -2.3482932948182267e-6 -1.1993651971575758e-6; … ; 0.0 -0.8629198617416143 … -0.994858584131923 -0.9962862870625954; 0.0 -0.0004970069812029743 … -1.6598054056182754e-6 5.365037639148973e-7], [0.0 -1.0456498357788244 … -0.9967143714511489 -0.9976300894886776; 0.0 0.0010225684979332794 … 1.8796300665924193e-6 1.4834216505166253e-6; … ; 0.0 -0.8632257507145933 … -0.9948591665807796 -0.9962855852825949; 0.0 -3.110287349859797e-5 … -7.726687739628701e-7 -5.323879362362227e-7], [0.0 -1.0029124807280965 … -0.9963253677142064 -0.9973510880199432; 0.0 -0.0003140314760316336 … 1.2603189495155598e-6 -3.134341234369009e-6; … ; 0.0 -0.8318020737727303 … -0.9945402782141182 -0.9960552051152544; 0.0 0.001033602936469924 … 2.548746084185538e-6 -4.4636627644090154e-7], [0.0 -0.11444056614764124 … -0.980977127397247 -0.9862184780308807; 0.0 0.0008440360209792299 … 2.779092394211702e-

In [14]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [15]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.10678624510315599 … -0.9806642820023019 -0.9859917441436443; 0.0 -0.00037569827549849494 … -1.437170107093308e-5 -1.0415118689831998e-5; … ; 0.0 -0.5945728642046613 … -0.9917294753940469 -0.9940095716255686; 0.0 0.002685405176205847 … 8.123533274187467e-6 6.051322588643271e-6], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [16]:
FEM.showDoFResults(q2, :qvec)

3

In [17]:
q3 = FEM.elementsToNodes(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.10678624510315599 … -0.9806642820023019 -0.9859917441436443; 0.0 -0.00037569827549849494 … -1.437170107093308e-5 -1.0415118689831998e-5; … ; 0.0 -0.5945728642046613 … -0.9917294753940469 -0.9940095716255686; 0.0 0.002685405176205847 … 8.123533274187467e-6 6.051322588643271e-6], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [18]:
FEM.showDoFResults(q3, :qvec)

4

In [19]:
err1 = FEM.fieldError(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.000375698275498465 … 1.4371701070770015e-5 1.0415118689766079e-5; 0.0 0.00037569827549849494 … 1.437170107093308e-5 1.0415118689831998e-5; … ; 0.0 0.00017852634485648888 … 1.8400294094500502e-6 1.4142528279581512e-6; 0.0 0.00015720930058077745 … 1.6203196053637958e-6 1.245383127249089e-6], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [20]:
FEM.showDoFResults(err1, :vector)

5

In [21]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [22]:
gmsh.finalize()